# Custom convnets for the DREAM ENCODE challenge

This document will outline the math for the non-standard components. 

## Ordinal likelihood

The nnet has a single univariate output $g \in R$ since I do not use a multitask output. The ordinal likelihood is defined for the Unbound, Ambigous and Bound classes as
\begin{align}
P(y=U|g)=\sigma(-g), \qquad P(y=A|g)=\sigma(1-g)-\sigma(-g), \qquad P(y=B|g)=1-\sigma(1-g)
\end{align}
where $\sigma$ is the logistic function. These probabilities are stacked in the `p` variable in the `tf_net`/`double_net` code. This setup is equivalent to sampling a new variable $g'$ which is logistic distributed with mean $g$ and unit scale, and setting $y=U$ if $g'<0$, $y=A$ if $g' \in [0,1]$ and $y=B$ if $g' > 1$. 

We can alternatively write the likelihood as
$$ P(y|g) = P(y=U|g)^{1[y=U]} P(y=A|g)^{1[y=A]} P(y=B|g)^{1[y=B]} $$
where $1[]$ the indicator function. Therefore
$$ \log P(y|g) = 1[y=U] \log P(y=U|g) + 1[y=A] \log P(y=A|g)^{} + 1[y=B] \log P(y=B|g) $$
which is the quantity calculated in the line
```{python}
neg_like=-( y * np.log(p + 1.0e-20) ).sum()
```
(note the + 1.0e-20 for numerical stability). Rows of `y` and `p` correspond to samples in the minibatch, columns correspond to the three classes. 

## Down-sampling of the negative set

The vast majority of the labels for any of the TFs are U(nbound). To speed up training in each epoch I only sample each `U` region with probability R=`unbound_rate=0.01`. To maintain an unbiased estimate of the gradient we modify the likelihood to be
$$ \frac1R 1[y=U] \log P(y=U|g) + 1[y=A] \log P(y=A|g)^{} + 1[y=B] \log P(y=B|g) $$

## Simultaneous analysis of the forward and reverse complement

TF ChIP-seq is not strand specific so the presence of motifs on either the forward or reverse complement (RC) contribute to the probability of binding. To account for this the `double_net` model runs the _same_ nnet on the forward and RC (including appropriately switching the Digital Genome Footprinting data). I chose to use the `max` of the output of these two copies of the nnet, since this is analogous to a logical OR: if we see an appropriate signal in _either_ direction then we expect to see binding. 